In [1]:
import ccxt
import pandas as pd
import numpy as np
import talib
import torch
import torch.nn as nn
import joblib
from datetime import datetime, timedelta


In [2]:
# -------------------------
# CONFIG
# -------------------------
API_KEY = ""      
API_SECRET = ""

symbol = "BTC/USDT"
model_timeframe = "1h"   # model trained on 1-hour candles
price_timeframe = "1m"   # execution monitored on 1-minute candles
limit = 200              
window_size = 10         

FEATURES = ['RSI', 'EMA12', 'EMA26', 'MACD', 'Signal', 'Histogram', 'DEMA9', 'SMA', 'TSI', '%K', '%D']

SAVED_MODEL = "greg_tech_2.pth"
SCALER_FILE = "scaler.pkl"

min_tp_pct = 0.005       
stop_loss_pct = 0.01     
position_fraction = 0.1  
check_interval_seconds = 60
simulate_only = True     

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [3]:
class CryptoLSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim):
        super(CryptoLSTM, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
    
    def forward(self, x):
        out, (hn, cn) = self.lstm(x)
        out = out[:, -1, :]  # last timestep
        out = self.fc(out)
        return out


model = CryptoLSTM(input_dim=len(FEATURES), hidden_dim=128, num_layers=4, output_dim=3)
model.load_state_dict(torch.load(SAVED_MODEL))
model.to(device)
model.eval()

scaler = joblib.load(SCALER_FILE)


In [4]:
def compute_tsi(close, r1=25, r2=13):
    delta = close.diff()
    ema1 = delta.ewm(span=r1, adjust=False).mean()
    ema2 = ema1.ewm(span=r2, adjust=False).mean()

    abs_delta = delta.abs()
    abs_ema1 = abs_delta.ewm(span=r1, adjust=False).mean()
    abs_ema2 = abs_ema1.ewm(span=r2, adjust=False).mean()

    tsi = 100 * (ema2 / abs_ema2)
    return tsi


In [5]:
import ta
def compute_indicators(df):
    out = {}
    out["RSI"] = ta.momentum.RSIIndicator(df["close"], window=14).rsi()
    out["EMA12"] = df["close"].ewm(span=12, adjust=False).mean()
    out["EMA26"] = df["close"].ewm(span=26, adjust=False).mean()
    out["MACD"] = out["EMA12"] - out["EMA26"]
    out["Signal"] = out["MACD"].ewm(span=9, adjust=False).mean()
    out["Histogram"] = out["MACD"] - out["Signal"]
    out["DEMA9"] = talib.DEMA(df["close"].values, timeperiod=9)
    sma_window = 3
    out['SMA'] = ta.trend.sma_indicator(df['close'], window=sma_window)
    out['TSI'] = compute_tsi(df['close'])
    period = 14
    smooth_k = 3
    smooth_d = 3

    lowest_low = df["low"].rolling(period).min()
    highest_high = df["high"].rolling(period).max()

    out["%K"] = 100 * (df["close"] - lowest_low) / (highest_high - lowest_low)
    out["%K"] = out["%K"].rolling(smooth_k).mean()
    out["%D"] = out["%K"].rolling(smooth_d).mean()
    
    feat_df = pd.DataFrame(out, index=df.index)
    return feat_df


In [7]:
# -------------------------
# EXCHANGE
# -------------------------
exchange = ccxt.binance({'enableRateLimit': True})


In [8]:
 #-------------------------
# FETCH DATA
# -------------------------
def fetch_latest_ohlcv(symbol, timeframe, limit):
    ohlcv = exchange.fetch_ohlcv(symbol, timeframe=timeframe, limit=limit)
    df = pd.DataFrame(ohlcv, columns=['timestamp','open','high','low','close','volume'])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    df.set_index('timestamp', inplace=True)
    df.columns = [c.lower() for c in df.columns]
    return df

In [9]:
df = fetch_latest_ohlcv(symbol, model_timeframe, limit)

df.shape

(200, 5)

In [10]:
df.head()

,open,high,low,close,volume
timestamp,,,,,
2025-09-22 02:00:00,114273.44,114618.89,114150.00,114187.81,895.13507
2025-09-22 03:00:00,114187.80,114654.36,114187.80,114649.89,326.66818
2025-09-22 04:00:00,114649.89,114649.90,114397.32,114433.53,208.01066
2025-09-22 05:00:00,114433.53,114551.80,113623.00,113691.19,753.79586
2025-09-22 06:00:00,113691.18,113698.38,111800.00,112800.77,5199.12797


In [11]:
df.tail()

,open,high,low,close,volume
timestamp,,,,,
2025-09-30 05:00:00,114091.63,114205.17,113827.24,113904.00,698.31455
2025-09-30 06:00:00,113904.00,114084.22,113838.41,113891.64,527.62041
2025-09-30 07:00:00,113891.64,113994.61,113585.00,113699.23,745.45417
2025-09-30 08:00:00,113699.24,113741.37,113350.35,113451.89,653.46315
2025-09-30 09:00:00,113451.89,113451.89,112942.41,113057.57,655.66802


In [18]:
while True:
        # 1) Model prediction from 1h data
        df_ohlcv = fetch_latest_ohlcv(symbol, model_timeframe, limit)
        feat_df = compute_indicators(df_ohlcv).dropna()
        last_rows = feat_df[FEATURES].values[-window_size:]
        print(last_rows.shape)

(10, 11)
(10, 11)
(10, 11)
(10, 11)
(10, 11)
(10, 11)
(10, 11)
(10, 11)
(10, 11)
(10, 11)
(10, 11)
(10, 11)
(10, 11)
(10, 11)


KeyboardInterrupt: 